# EarthShape Phase 2: Overview of time-slice simulations

This notebook provides a brief overview of the simulated timeslices and some examples for working with the netcdf files produced by the postprocessing tooling for [EarthShape](http://earthshape.net/) Phase 2.

__________________
<img src="images/logo_earthshape.png" width="330" align="right">

Christian Werner  
christian.werner@kit.edu  
IMK-IFU, Karlsruhe Institute for Technology, Garmisch-Partenkirchen  
2019-06-01

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [22]:
from pathlib import Path
import sys
import xarray as xr
from IPython.display import Markdown

def check_version():
    def rtext(t): return '<span style="color: #ff0000">'+t+'</span>'
    if float('.'.join([str(x) for x in sys.version_info[0:2]] )) < 3.6:
        display(Markdown(rtext('This notebook is written for Python 3.6 and up!')))
        
check_version()

## Climate data

Climate data was provided by Sebastian Mutz (Uni Tübingen). The time slices are:

|Time-slice      |Code |Time    |CO$_2$|Status    |Comment                       |
|:---------------|:----|:-------|:-----|:---------|:-----------------------------|  
|Present Day     |e004 |.       |348   |NA        |used for bias correction only, p.d. (1978-1996)	e5 default	p.d. mean (1957-2000)	e5 default (CO2: 348ppm, CH4:1.65ppm) |
|Pre Industrial  |e007 |.       |280   |**Done**  |amip pre-industry	PI co2(280ppm),ch4(760ppb),n2o(270ppb)echam def          |
|Middle Holocene |e008 |.       |280   |NA        |e5 default	AWI_MH_clim_sst	co2(280ppm),ch4(650ppb),n2o(270ppb)              |
|LGM             |e009 |.       |185   |**Done**  |reconstructed LGM, AWI_LGM_clim_sst, LGM co2(185ppm),ch4(350ppb),n2o(200ppb)  |
|Pliocene        |e010 |.       |405   |**Done**  |reconsructed from PRISM datasets	reconstructed from PRISM dataset, Pliocene co2(405ppm), others using PI ch4(760ppb),n2o(270ppb)         |
|Late Miocene    |e012 |11-7 ma |450   |NA        |Greenland ice removed; hudson bay filled with low elevation (5meter); Austria moved southeast wards for 5 grid boxes;Vegetation from Michel et al. 2011;south antartic ice 1/4 of the present; rest are the same as PI, knorr and Lohmann 2014, ICE Quarter CO2 450ppm	CO2 (450ppm) others PI values       |
|Middle Miocene  |e011 |14 ma   |278   |**Done**  |Greenland ice removed; hudson bay filled with low elevation (5meter); Austria moved southeast wards for 5 grid boxes;Vegetation from Michel et al. 2011; rest are the same as PI	Knorr and Lohmann 2014, ICE Full CO2 278ppm run	CO2(278ppm) others PI values                            |
